# Video Processing

This notebook loads the saved model and processes detection on a video.

In [1]:
import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import imageio

imageio.plugins.ffmpeg.download()

from moviepy.editor import VideoFileClip
import PIL.Image
import time
import numpy as np
import cv2

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


`VideoFileClip` needs an instance to hold the test generator. This `VideoProcessor` instance retains the generator instance and also provides a processing function.

In [2]:
class VideoProcessor():
    def __init__(self, threshold):
        self.model = keras.models.load_model('./snapshots/resnet50_csv_18.h5', custom_objects=custom_objects)
        # create image data generator object for the preprocessing functionality
        generator = keras.preprocessing.image.ImageDataGenerator()
        self.csv_generator = CSVGenerator(
            './data/test.csv',
            './data/classes.csv',
            generator,
            batch_size=1,
        )
        self.threshold = threshold
        
    def process_image(self, image):
        # copy to draw on
        draw = image.copy()

        # preprocess image for network
        image = self.csv_generator.preprocess_image(image)
        image, scale = self.csv_generator.resize_image(image)

        # process image
        start = time.time()
        _, _, detections = self.model.predict_on_batch(np.expand_dims(image, axis=0))

        # compute predicted labels and scores
        predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
        scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

        # correct for image scale
        detections[0, :, :4] /= scale

        # visualize detections
        for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
            if score < self.threshold:
                continue
            b = detections[0, idx, :4].astype(int)
            cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 3)
            caption = "{} {:.3f}".format(self.csv_generator.label_to_name(label), score)
            cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 0), 3)
            cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2)

        return draw

In [3]:
def process_video(processor, input_path, output_path):
    clip = VideoFileClip (input_path)
    result = clip.fl_image(processor.process_image)
    %time result.write_videofile (output_path, audio=False)

In [5]:
ironman_processor = VideoProcessor(threshold=0.10)
process_video(ironman_processor, './videos/ironman.mp4', './videos/ironman_result.mp4')

[MoviePy] >>>> Building video ./videos/ironman_result.mp4
[MoviePy] Writing video ./videos/ironman_result.mp4


100%|██████████| 1131/1131 [03:35<00:00,  5.22it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./videos/ironman_result.mp4 

CPU times: user 1min 4s, sys: 3.1 s, total: 1min 7s
Wall time: 3min 35s


In [6]:
selfie_processor = VideoProcessor(threshold=0.25)
process_video(selfie_processor, './videos/selfie.mp4', './videos/selfie_result.mp4')

[MoviePy] >>>> Building video ./videos/selfie_result.mp4
[MoviePy] Writing video ./videos/selfie_result.mp4


100%|██████████| 802/802 [03:17<00:00,  4.13it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./videos/selfie_result.mp4 

CPU times: user 1min 6s, sys: 3.21 s, total: 1min 9s
Wall time: 3min 21s
